In [ ]:
!pip uninstall torch torchvision torchaudio
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Load Data

In [ ]:
# ==============================================================================
# Cell 2: Data Loading & Transformation (CIFAR-10)


data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224), # Resize the tiny 32x32 image to the size ResNet expects
        transforms.RandomHorizontalFlip(), # A standard, non-destructive augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224), # Just resize for validation, no cropping needed
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,
                                             download=True,
                                             transform=data_transforms['train'])

val_dataset = torchvision.datasets.CIFAR10(root='./data',
                                           train=False,
                                           download=True,
                                           transform=data_transforms['val'])

image_datasets = {'train': train_dataset, 'val': val_dataset}


dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# We want to be able to train our model on an `accelerator`
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")
print(f"Classes: {class_names}")

print("\nCell 2 Execution Complete: Data pipeline re-engineered for CIFAR-10.")



# View Images

In [ ]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

# Train Model

In [ ]:
def train_model(model, criterion, optimizer, scheduler, save_path, num_epochs=25, patience=5, min_delta=0.001):
    """
    Trains a model and saves the best performing weights to a specified path.
    Includes an early stopping mechanism.
    """
    since = time.time()

    # Ensure the directory for the save_path exists
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    best_model_params_path = save_path
    best_acc = 0.0

    # --- Early Stopping Parameters ---
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # --- Early Stopping Logic ---
            if phase == 'val':
                # Check if the model has improved by at least min_delta
                if epoch_acc > best_acc + min_delta:
                    best_acc = epoch_acc
                    epochs_no_improve = 0  # Reset patience counter
                    torch.save(model.state_dict(), best_model_params_path)
                    print(f"New best model saved to {best_model_params_path} with accuracy: {best_acc:.4f}")
                else:
                    epochs_no_improve += 1

        # Check if training should be stopped
        if epochs_no_improve >= patience:
            print(f"\nEarly stopping triggered after {patience} epochs with no improvement.")
            break

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    model.load_state_dict(torch.load(best_model_params_path))
    return model

print("Cell 4 Execution Complete: train_model function is defined with Early Stopping.")



#Visualize Model Predictions

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

# Finetune the ConvNet

In [ ]:
# --- FINETUNING THE CONVNET FOR CIFAR10 ---


# 1. Load the pretrained ResNet18 architecture and weights
model_ft = models.resnet18(weights='IMAGENET1K_V1')

# 2. Extract the number of input features for the final layer
num_ftrs = model_ft.fc.in_features

# 3. Re-engineer the final classification layer for the 10 CIFAR10 classes
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

# 4. Transfer the model to the appropriate computational substrate (GPU or CPU)
model_ft = model_ft.to(device)

# 5. Define the loss function (criterion)
criterion = nn.CrossEntropyLoss()

# 6. Define the optimizer, ensuring it targets all parameters of the fine-tuned model
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# 7. Define the learning rate scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

print("Model_ft successfully initialized and configured.")
print(f"Model is on device: {next(model_ft.parameters()).device}")

## Train and evaluate

In [ ]:
# 1a. Define the permanent archival path in your Google Drive for the fine-tuned model.
finetuned_model_save_path = "/content/drive/MyDrive/Colab_Models/cifar10_resnet18_finetuned.pt"

# 1b. Call the training function with 'patience' set to halt the process.
print("\n--- INITIATING PROTOCOL 1: FINETUNING (WITH EARLY STOPPING) ---")
model_ft = train_model(model_ft,
                       criterion,
                       optimizer_ft,
                       exp_lr_scheduler,
                       save_path=finetuned_model_save_path,
                       num_epochs=25,
                       patience=5, # <-- Stops unnecessary training
                       min_delta=0.001)

print("\nProtocol 1 Execution Complete: Fine-tuning finished.")

In [ ]:
visualize_model(model_ft)

# ConvNet as fixed feature extractor

In [ ]:
# --- Protocol 2: ConvNet as a Fixed Feature Extractor ---

print("\n--- INITIATING PROTOCOL 2: FEATURE EXTRACTION (WITH EARLY STOPPING) ---")

# 2a. Load a fresh pretrained model
model_conv = models.resnet18(weights='IMAGENET1K_V1')

# Freeze all the network's parameters
for param in model_conv.parameters():
    param.requires_grad = False

# Re-engineer the final layer for the 10 CIFAR10 classes.
# Parameters of newly constructed modules have requires_grad=True by default.
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

# Transfer the model to the appropriate computational substrate
model_conv = model_conv.to(device)

# Define the loss function
criterion_conv = nn.CrossEntropyLoss()

# Define the optimizer to ONLY train the new final layer
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.01, momentum=0.9)

# Define the learning rate scheduler
exp_lr_scheduler_conv = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

## Train and Evaluate

In [ ]:
# 2b. Define the archival path for the feature extractor model.
feature_extractor_model_save_path = "/content/drive/MyDrive/Colab_Models/cifar10_resnet18_feature_extractor.pt"

# 2c. Call the training function for the feature extractor model.
model_conv = train_model(model_conv,
                         criterion_conv,
                         optimizer_conv,
                         exp_lr_scheduler_conv,
                         save_path=feature_extractor_model_save_path,
                         num_epochs=25,
                         patience=5, # <-- Stops unnecessary training
                         min_delta=0.001)

print("\nProtocol 2 Execution Complete: Feature extraction training finished.")
print("\nCell 6 Execution Complete: Both models trained and best versions loaded.")

# Inference on custom images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def un_normalize(tensor):
    """Reverses the normalization on a tensor to make it displayable."""
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    # Create a new tensor to avoid modifying the original in place
    un_normalized_tensor = tensor.clone()
    for t, m, s in zip(un_normalized_tensor, mean, std):
        t.mul_(s).add_(m)
    return un_normalized_tensor

def visualize_model_prediction_tensor(model, img_tensor, true_label, ax=None):
    """Visualizes a model's prediction for a single image tensor."""
    if ax is None:
        fig, ax = plt.subplots()

    # --- Model Prediction Logic ---
    was_training = model.training
    model.eval()

    with torch.no_grad():
        # Add a batch dimension and send to device
        outputs = model(img_tensor.unsqueeze(0).to(device))
        _, preds = torch.max(outputs, 1)
        predicted_class = class_names[preds[0]]

    # --- Visualization Logic ---
    # Un-normalize the image tensor before displaying
    display_tensor = un_normalize(img_tensor.cpu())

    # Transpose from (C, H, W) to (H, W, C) for matplotlib
    ax.imshow(display_tensor.permute(1, 2, 0))
    ax.axis('off')
    ax.set_title(f'True: {true_label}\nPredicted: {predicted_class}')

    model.train(mode=was_training)
    return ax, predicted_class



In [ ]:
# Get a single image and label from the validation set
img, label = val_dataset[200]
true_class_name = class_names[label]

# Call the new function
visualize_model_prediction_tensor(model_conv, img, true_class_name)
plt.ioff()
plt.show()